<a href="https://colab.research.google.com/github/skywalker0803r/mxnet_course/blob/master/mxnet_LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

看一下gpu是否正常work

In [1]:
!nvidia-smi

Sat Nov 30 15:08:52 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

安裝mxnet-cu100 d2lzh

In [0]:
#!pip install mxnet-cu100 d2lzh

# Lenet model

![alt text](https://miro.medium.com/max/3800/1*tdLQdtGy8CopdIRWhqBLkw.png)

In [0]:
import d2lzh as d2l
from d2lzh import *
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn
import time

net = nn.Sequential()
net.add(nn.Conv2D(channels=6, kernel_size=5, activation='sigmoid'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Conv2D(channels=16, kernel_size=5, activation='sigmoid'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Dense(120, activation='sigmoid'),
        nn.Dense(84, activation='sigmoid'),
        nn.Dense(10))

# check model layers shape

In [4]:
X = nd.random.uniform(shape=(1, 1, 32, 32))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv0 output shape:	 (1, 6, 28, 28)
pool0 output shape:	 (1, 6, 14, 14)
conv1 output shape:	 (1, 16, 10, 10)
pool1 output shape:	 (1, 16, 5, 5)
dense0 output shape:	 (1, 120)
dense1 output shape:	 (1, 84)
dense2 output shape:	 (1, 10)


# 获取数据和训练模型

In [0]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size,resize=(32,32))

# set ctx

In [0]:
ctx = mx.gpu()

# training

In [7]:
lr, num_epochs = 0.9, 5
net.initialize(force_reinit=True ,ctx=ctx ,init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on gpu(0)
epoch 1, loss 2.3194, train acc 0.098, test acc 0.154, time 6.0 sec
epoch 2, loss 1.6970, train acc 0.344, test acc 0.658, time 5.8 sec
epoch 3, loss 0.8856, train acc 0.652, test acc 0.708, time 6.0 sec
epoch 4, loss 0.7118, train acc 0.720, test acc 0.730, time 6.3 sec
epoch 5, loss 0.6353, train acc 0.749, test acc 0.774, time 6.2 sec
